In [1]:
#DAAE trained on celebA faces
import sys
sys.path.append('../')

from function import prep_data, make_new_folder, plot_losses, save_input_args, shift_x, plot_norm_losses
from dataload import CELEBA 
from models import DAE, DIS_Z, LINEAR_SVM

import torch
from torch import optim

from torchvision.utils import make_grid, save_image
from torchvision import transforms, datasets

import argparse

from time import time

import os
from os.path import join

import numpy as np

import matplotlib
from matplotlib import pyplot as plt

/home/xyang18/miniconda3/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Args:
    def __init__(self):
        self.root = '../data'
        self.batchSize = 64
        self.maxEpochs = 100
        self.nz = 200
        self.lr = 1e-4
        self.fSize = 64
        self.outDir = '../Experiments/DAAE1000/'
        self.commit = 'xxx'
        self.alpha = 1.0
        self.sigma = 0.1
        self.M = 5
        self.loss = 'BCE' #'BCE'
        self.loadDAE = False
        self.loadSVM = False
        self.load_DAE_from = None
        self.evalMode = False
        self.comment = ''
        self.momentum = 0.1
        self.c = 0.01
        self.svmLR = 1e-4
        self.Ntest = 100
        self.gpuNo = 3
        self.multimodalZ = False


In [3]:
# def get_args():
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--root', default='/data', type=str) #data/datasets/LabelSwap
#     parser.add_argument('--batchSize', default=64, type=int)
#     parser.add_argument('--maxEpochs', default=10, type=int)
#     parser.add_argument('--nz', default=200, type=int)
#     parser.add_argument('--lr', default=1e-3, type=float)
#     parser.add_argument('--fSize', default=64, type=int)  # multiple of filters to use
#     parser.add_argument('--outDir', default='../../Experiments/DAAE1000/', type=str)
#     # parser.add_argument('--commit', required=True, type=str)
#     parser.add_argument('--commit', default='xxx', type=str)

#     parser.add_argument('--alpha', default=1.0, type=float)  # weight on the adversarial cost
#     parser.add_argument('--sigma', default=0.1, type=float)  # noise level
#     parser.add_argument('--M', default=5, type=int)  #number of sampling iterations
#     parser.add_argument('--loss', default='BCE', type=str) #'BCE' or 'MSE' currently supported
#     parser.add_argument('--loadDAE', action='store_true')
#     parser.add_argument('--loadSVM', action='store_true')
#     parser.add_argument('--load_DAE_from', default=None, type=str)
#     parser.add_argument('--evalMode', action='store_true')
#     parser.add_argument('--comment', type=str)
#     parser.add_argument('--momentum', default=0.9, type=float) 
#     parser.add_argument('--c', type=float, default=0.01) #for training the linearSVM for eval
#     parser.add_argument('--svmLR', type=float, default=1e-4)

#     parser.add_argument('--Ntest', default=100, type=int)
#     # parser.add_argument('--gpuNo', required=True, type=int)
#     parser.add_argument('--gpuNo', default=0, type=int)


#     return parser.parse_args()

def build_dis(dae, multimodalZ):
    if not multimodalZ:
        print('\n ** USING NORMAL PRIOR **')
        prior = dae.norm_prior
        NZ = opts.nz
    else:
        print('\n ** USING MULTIMODAL PRIOR **')
        prior = dae.multi_prior
        NZ = 2
    dis = DIS_Z(nz=NZ, prior=prior)

    return dis, NZ

def svm_score(svm, y, x=None, enc=None, dae=None):
    '''
    EITHER
        take a data sample x AND a dae
    OR
        take an encoding
    and apply SVM and get score

    '''
    assert (x is not None) or (enc is not None)
    if enc is None:
        assert dae is not None
        enc = dae.encode(x)
    output = svm.forward(enc)
    score = svm.binary_class_score(output, y)
    return score

def eval_mode(dae, exDir, M, testLoader, svm=None):
    f = open(join(exDir, 'outputs.txt'), 'w')

    ## reconstruction error ##
    recError = []
    print('calculating reconstruction error...')
    for i, data in enumerate(testLoader):
        x, y = prep_data(data, useCUDA=dae.useCUDA)
        zTest = dae.encode(x)
        recTest = dae.decode(zTest)
        # recError.append(dae.rec_loss(recTest, x).data[0])
        recError.append(dae.rec_loss(recTest, x).detach().cpu().numpy())
    meanRecError = np.mean(recError)
    f.write('mean reconstruction error (non-corrupted): %0.5f' % (meanRecError))


    #eval samples ##TODO

    ## representation robustness (shift) ##
    print('performing robustness plot...')
    maxShift = x.size(2)//2
    step = 4
    axis = range(-maxShift, maxShift, step)
    robustnessMap = torch.Tensor(maxShift*2//step, maxShift*2//step).fill_(0)
    classMap = torch.Tensor(maxShift*2//step, maxShift*2//step).fill_(0)
    x, y = prep_data(iter(testLoader).next(), useCUDA=dae.useCUDA)  #take a batch of samples
    allShifts=[]
    enc00 = dae.encode(x)
    for j, dx in enumerate(axis):
        for i, dy in enumerate(axis):
            xShift = shift_x(x, dy, dx)
            encDxDy = dae.encode(xShift)
            # diff = [(torch.dot(encDxDy[k], enc00[k])/ (torch.norm(encDxDy[k])*torch.norm(enc00[k]))).data[0] for k in range(encDxDy.size(0))]
            # diff = [torch.dot(encDxDy[k], enc00[k]).data[0]/ ((torch.norm(encDxDy[k])*torch.norm(enc00[k])).data[0] + 1e-6) for k in range(encDxDy.size(0))]
            diff = [torch.dot(encDxDy[k], enc00[k]).detach().cpu().numpy()/ ((torch.norm(encDxDy[k])*torch.norm(enc00[k])).detach().cpu().numpy() + 1e-6) for k in range(encDxDy.size(0))]
            robustnessMap[j,i] = np.mean(diff)
            # classMap[j,i] = svm_score(svm, y, enc=encDxDy).data[0]
            classMap[j,i] = svm_score(svm, y, enc=encDxDy).detach().cpu()

            allShifts.append(xShift[0].cpu().data.numpy())

    print('saving images...')
    print(type(allShifts), np.shape(allShifts))
    save_image(torch.Tensor(np.asarray(allShifts)), join(exDir,'shiftImages.png'), nrow=16)
    print(robustnessMap)

    print('save maps as numpy array...')
    np.save(join(exDir, 'classMap.npy'), classMap.numpy())
    np.save(join(exDir, 'shiftMap.npy'), robustnessMap.numpy())

    # plot shift robustenss map
    fig0 = plt.figure()
    print(robustnessMap.min(), robustnessMap.max(), robustnessMap.size())
    f.write('\nrobustness min: %0.5f, max: %0.5f' % (robustnessMap.min(), robustnessMap.max()))
    plt.imshow(robustnessMap.numpy(), extent=[-maxShift, maxShift, -maxShift, maxShift], vmin=0.0, vmax=1.0)
    plt.xlabel('DX')
    plt.ylabel('DY')
    plt.title('Robustness to shifts in x and y')
    plt.colorbar()
    plt.savefig(join(exDir, 'ShiftRobustness.png'))

    #Plot shift robusteness classification map
    fig1 = plt.figure()
    f.write('\nshift robustenss accuracy min: %0.5f, max: %0.5f' % (classMap.min(), classMap.max()))
    f.write('\nAccuray Volume (sum of elements in accuracy shift map): %0.5f' % (np.clip(classMap.numpy(), 0.5, 1).sum()))
    plt.imshow(classMap.numpy(), extent=[-maxShift, maxShift, -maxShift, maxShift], vmin=0.5, vmax=1.0)
    plt.xlabel('DX')
    plt.ylabel('DY')
    plt.title('Classiciation Robustness to shifts in x and y')
    plt.colorbar()
    plt.savefig(join(exDir, 'ClassificationShiftRobustness.png'))

    ## Compare histograms for enc, z_samples and encCorr
    fig2 = plt.figure()
    # nEnc, bEnc, _ = plt.hist(enc00.cpu().data.numpy().flatten(), 100, normed=True)
    nEnc, bEnc, _ = plt.hist(enc00.cpu().data.numpy().flatten(), 100, density=True, stacked=True)
    xcorr = dae.corrupt(x)
    encCorr = dae.encode(xcorr)
    # nEncCorr, bEncCorr, _ = plt.hist(encCorr.cpu().data.numpy().flatten(), 100, normed=True)
    # nNorm, bNorm, _ = plt.hist(dae.sample_z(10000).cpu().data.numpy().flatten(), 100, normed=True)
    nEncCorr, bEncCorr, _ = plt.hist(encCorr.cpu().data.numpy().flatten(), 100, density=True, stacked=True)
    nNorm, bNorm, _ = plt.hist(dae.sample_z(10000).cpu().data.numpy().flatten(), 100, density=True, stacked=True)
    fig3 = plt.figure()
    plt.plot(bEnc[1:], nEnc, label='encoding')
    plt.plot(bEncCorr[1:], nEncCorr, label='corrupted encoding')
    plt.plot(bNorm[1:], nNorm, label='Normal')
    plt.title('Comparing Encodings')
    plt.xlabel('Value')
    plt.ylabel('pdf')
    plt.legend()
    plt.savefig(join(exDir, 'HisEnc.png'))

    #save all histograms:
    np.save(join(exDir, 'HistEnc.npy'), [nEnc,bEnc])
    np.save(join(exDir), 'HistEncCorr.npy', [nEncCorr, bEncCorr])
    np.save(join(exDir), 'prior.npy', [nNorm, bNorm])

    #sampling
    print('sampling...')
    sampleDir = join(exDir,'FinalSamples')
    try:
        os.mkdir(sampleDir)
    except OSError: print('file alread exists')
    dae.sample_x(opts.M, sampleDir)

    ## classification test score
    if svm is not None:
        'Do classification'
        testScore = 0
        for i, data in enumerate(testLoader):
            x, y = prep_data(data, useCUDA=svm.useCUDA)
            score = svm_score(svm, y, x=x, dae=dae) 
            testScore+=score
    testScore /= (i+1)
    # f.write('\nSVM classification (test) score:'+str(testScore.data[0]))
    f.write('\nSVM classification (test) score:'+str(testScore.detach().cpu().numpy()))
    f.close()




def train_svm(dae, svm, trainLoader, testLoader, exDir, lr):
    '''
    Data y is [0,1]
    For training SVM must be -1, 1
    To eval data put back to [0,1]

    To get loss use [-1,1] for train and test
    To get score use [0,1] for train and test
    '''
    print('training svm...')
    dae.eval()
    optimSVM = optim.SGD(svm.parameters(), lr=lr) #optimizer  

    f = open(join(exDir, 'svmOpts.txt'), 'w')
    f.write('smvLR: %0.5f\nc: %0.5f\n' % (lr, svm.c))
    f.close() 


    svmLoss = {'train':[], 'test':[]}
    for epoch in range(opts.maxEpochs):
        epochLoss_svm=0
        svm.train()
        T = time()
        for i, data in enumerate(trainLoader):
            x, y = prep_data(data, useCUDA=svm.useCUDA)  #prep data as a var
            inputs = dae.encode(x)  #get encodings as input
            output = svm.forward(inputs)  #get output
            loss = svm.loss(output, y * 2 - 1)  #calc loss 
            optimSVM.zero_grad()  #zero grad
            loss.backward()  #backwards
            optimSVM.step()  #step
            # epochLoss_svm+=loss.data[0]
            epochLoss_svm+=loss.detach().cpu().numpy()
            if i%100 == 0:
                print('[%d, %i] loss: %0.5f, time: %0.3f' % (epoch, i, epochLoss_svm/(i+1), time() - T))
        svm.save_params(exDir)
        svmLoss['train'].append(epochLoss_svm/(i+1))

        #test loss:
        svm.eval()
        xTest, yTest = prep_data(iter(testLoader).next(), useCUDA=svm.useCUDA)
        testInputs = dae.encode(xTest)
        testOutputs = svm.forward(testInputs)
        testLoss = svm.loss(testOutputs, yTest * 2 - 1)
        # svmLoss['test'].append(testLoss.data[0])
        svmLoss['test'].append(testLoss.detach().cpu().numpy())

        if epoch > 1:
            plot_losses(svmLoss, exDir=exDir, epochs=epoch+1, title='SVM_loss')

        #Do classification
        testScore = svm.binary_class_score(testOutputs, yTest) #has threshold as zero for testOutputs in [-1,1]
        trainScore = svm.binary_class_score(output, y) #has threshold as zero for output in [-1,1]
        f = open(join(exDir, 'svm.txt'), 'w')
        # f.write('trainScore: %f \ntestScore: %f ' \
        #  % (trainScore.mean().data[0], testScore.mean().data[0]))
        f.write('trainScore: %f \ntestScore: %f ' \
         % (trainScore.mean().detach().cpu().numpy(), testScore.mean().detach().cpu().numpy()))
        f.close()

    return svm

In [4]:
# opts = get_args()


In [5]:
# if __name__=='__main__':
# opts = get_args()
opts = Args()

#Load data
print('Prepare data loaders...')
transform = transforms.Compose([transforms.ToTensor(), transforms.RandomHorizontalFlip()])
trainDataset = CELEBA(root=opts.root, train=True, transform=transforms.ToTensor())
trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=opts.batchSize, shuffle=True)

testDataset = CELEBA(root=opts.root, train=False, transform=transforms.ToTensor())
testLoader = torch.utils.data.DataLoader(testDataset, batch_size=opts.batchSize, shuffle=False)
print('Data loaders ready.')

Prepare data loaders...
31
(202499,) (202499, 64, 64, 3)
[0. 1.]
31
Data loaders ready.


In [6]:
#Create model
dae = DAE(nz=opts.nz, imSize=64, fSize=opts.fSize, sigma=opts.sigma, multimodalZ=opts.multimodalZ) #sigma=level of corruption
dis, NZ = build_dis(dae=dae, multimodalZ=opts.multimodalZ)
svm = LINEAR_SVM(nz=NZ, c=opts.c) #model

if dae.useCUDA:
    torch.cuda.set_device(opts.gpuNo)
    print('using gpu:', torch.cuda.current_device())
    dae.cuda()
    dis.cuda()
    svm.cuda()


 ** USING NORMAL PRIOR **
using gpu: 3


In [7]:
if opts.loadDAE:  #should load DAE if in eval mode
    print('loading DAE...')
    dae.load_params(opts.load_DAE_from)

if opts.loadSVM:
        svm.load_params(opts.load_DAE_from) #use SVM @ same location as DAE [may not be one there]

if opts.evalMode & (not opts.loadSVM):  #to train an SVM for eval
        svm = train_svm(dae=dae, svm=svm, trainLoader=trainLoader, testLoader=testLoader, exDir=opts.load_DAE_from, lr=opts.svmLR)

if opts.evalMode:
    assert opts.loadDAE == True
    eval_mode(dae, opts.load_DAE_from, opts.M, testLoader, svm=svm)
    exit()
else:
    #Create a folder for this experiment
    exDir = make_new_folder(opts.outDir)
    print('Outputs will be saved to:',exDir)
    save_input_args(exDir, opts)  #save training opts

Outputs will be saved to: ../Experiments/DAAE1000/Ex_68


0

In [8]:
#Create optimizers
optimDAE = optim.RMSprop(dae.parameters(), lr = opts.lr)
optimDIS = optim.RMSprop(dis.parameters(), lr = opts.lr, momentum=opts.momentum)

#Keeping track of training
losses = {'enc': [], 'rec': [], 'dis':[], 'test rec':[]}

In [32]:
xTest, yTest = prep_data(iter(testLoader).next(), useCUDA=dae.useCUDA)


In [33]:
noise = torch.round(torch.randn(xTest.size()).type_as(xTest))

In [34]:
noise = torch.randint(0, 2, xTest.size()).type_as(xTest)

In [35]:
print(xTest)

tensor([[[[1.0000, 1.0000, 1.0000,  ..., 0.9843, 0.9765, 0.9765],
          [0.9961, 0.9961, 0.9961,  ..., 0.9765, 0.9804, 0.9725],
          [0.9922, 0.9922, 0.9922,  ..., 0.9647, 0.9843, 0.9804],
          ...,
          [0.6471, 0.6392, 0.5451,  ..., 0.7137, 0.7294, 0.7333],
          [0.6824, 0.6392, 0.6078,  ..., 0.6745, 0.6706, 0.6902],
          [0.6863, 0.6784, 0.6941,  ..., 0.7608, 0.7098, 0.6824]],

         [[0.9216, 0.9216, 0.9216,  ..., 0.9490, 0.9255, 0.9451],
          [0.9216, 0.9216, 0.9216,  ..., 0.9529, 0.9333, 0.9490],
          [0.9294, 0.9294, 0.9294,  ..., 0.9529, 0.9333, 0.9451],
          ...,
          [0.3882, 0.4000, 0.3294,  ..., 0.5255, 0.5294, 0.5294],
          [0.4275, 0.3882, 0.3608,  ..., 0.4510, 0.4667, 0.4902],
          [0.4275, 0.4275, 0.4471,  ..., 0.5490, 0.5137, 0.4824]],

         [[0.7765, 0.7765, 0.7765,  ..., 0.8627, 0.8745, 0.9020],
          [0.7765, 0.7765, 0.7765,  ..., 0.8745, 0.8784, 0.9020],
          [0.7765, 0.7765, 0.7765,  ..., 0

In [10]:
save_image(dae.corrupt(xTest.data), join(exDir, 'corrupted.png'))


In [18]:
torch.min(xTest[0])

tensor(0., device='cuda:3')

In [ ]:
#Start training
with torch.autograd.set_detect_anomaly(True):
    for e in range(opts.maxEpochs):

        epochEncLoss=0
        epochRecLoss=0
        epochDisLoss=0

        for i, data in enumerate(trainLoader):
            # print(i)

            T = time()

            dae.train()
            dis.train()

            x, y = prep_data(data, useCUDA=dae.useCUDA)

            # get outputs
            zFake, xRec = dae.forward(x)

            # clac losses
            recLoss = dae.rec_loss(xRec, x, loss=opts.loss)  #loss='BCE' or 'MSE'
            
            optimDAE.zero_grad()
            recLoss.backward()
            optimDAE.step()            
            
            
            zFake, xRec = dae.forward(x)    
            
            disLoss = dis.dis_loss(zFake)


            
            #do updates

            optimDIS.zero_grad()
            disLoss.backward()
            optimDIS.step()
            
            encLoss = dis.gen_loss(zFake)
            
            optimDAE.zero_grad()
            encLoss.backward()
            optimDAE.step()                   
                        
            # encLoss = dis.gen_loss(zFake)
            # daeLoss = recLoss + opts.alpha * encLoss

            # print(recLoss)
            # print(encLoss)
            # print(disLoss)            
            

            
            # print(encLoss.data)

            # storing losses for plotting later
            # epochEncLoss+=encLoss.data[0]
            # epochRecLoss+=recLoss.data[0]
            # epochDisLoss+=disLoss.data[0]
            epochEncLoss+=encLoss.detach().cpu().numpy()
            epochRecLoss+=recLoss.detach().cpu().numpy()
            epochDisLoss+=disLoss.detach().cpu().numpy()
            
            if i%100 == 0:
                # print('[%d, %d] enc: %0.5f, rec: %0.5f, dis: %0.5f, time: %0.3f' % (e, i, encLoss.data[0], recLoss.data[0], disLoss.data[0], time() - T))
                print('[%d, %d] enc: %0.5f, rec: %0.5f, dis: %0.5f, time: %0.3f' % (e, i, encLoss.detach().cpu().numpy(), recLoss.detach().cpu().numpy(), disLoss.detach().cpu().numpy(), time() - T))

        # storing losses for plotting later
        losses['enc'].append(epochEncLoss/i)
        losses['rec'].append(epochRecLoss/i)
        losses['dis'].append(epochDisLoss/i)

        #### Test
        dae.eval()
        dis.eval()

        #get test outuputs and losses
        xTest, yTest = prep_data(iter(testLoader).next(), useCUDA=dae.useCUDA)
        zTest, recTest = dae.forward(xTest)  #N.B. corruption in here
        recLossTest = dae.rec_loss(recTest, xTest)

        #Plot losses
        # losses['test rec'].append(recLossTest.data[0])
        losses['test rec'].append(recLossTest.detach().cpu().numpy())

        if e > 0: #only one point for test rec otherwise
            plot_losses(losses, exDir, epochs=e+1)
            plot_norm_losses(losses, exDir)

        #save parameters
        dae.save_params(exDir)
        dis.save_params(exDir)

        #Save images of original and rec
        save_image(xTest.data, join(exDir, 'original.png'))
        save_image(recTest.data, join(exDir, 'rec.png'))

        #Save samples
        sampleDir = join(exDir,'epoch_'+str(e))
        os.mkdir(sampleDir)
        print('sample dir:', sampleDir)
        dae.sample_x(opts.M, sampleDir)



/home/xyang18/miniconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


[0, 0] enc: 0.61824, rec: 0.68935, dis: 0.71291, time: 0.523
[0, 100] enc: 0.69863, rec: 0.67806, dis: 0.74379, time: 0.092
[0, 200] enc: 0.73390, rec: 0.67498, dis: 0.70315, time: 0.085
[0, 300] enc: 0.70304, rec: 0.66811, dis: 0.73000, time: 0.093
[0, 400] enc: 0.70626, rec: 0.62732, dis: 0.69855, time: 0.088
[0, 500] enc: 0.70431, rec: 0.58283, dis: 0.69957, time: 0.086
[0, 600] enc: 0.73027, rec: 0.59865, dis: 0.66890, time: 0.095
[0, 700] enc: 0.81091, rec: 0.59463, dis: 0.66671, time: 0.105
[0, 800] enc: 0.83953, rec: 0.58821, dis: 0.70528, time: 0.091
[0, 900] enc: 0.84110, rec: 0.58232, dis: 0.67090, time: 0.092
[0, 1000] enc: 0.75725, rec: 0.57567, dis: 0.70241, time: 0.097
[0, 1100] enc: 0.84924, rec: 0.56851, dis: 0.70100, time: 0.091
[0, 1200] enc: 0.82635, rec: 0.58053, dis: 0.67261, time: 0.098
[0, 1300] enc: 0.87319, rec: 0.57217, dis: 0.65398, time: 0.093
[0, 1400] enc: 0.90620, rec: 0.57819, dis: 0.71810, time: 0.093
[0, 1500] enc: 0.89227, rec: 0.55981, dis: 0.71501, 

/home/xyang18/miniconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


[2, 0] enc: 0.73030, rec: 0.54588, dis: 0.69283, time: 0.087
[2, 100] enc: 0.72379, rec: 0.54557, dis: 0.69937, time: 0.094
[2, 200] enc: 0.73624, rec: 0.53876, dis: 0.69101, time: 0.091
[2, 300] enc: 0.75258, rec: 0.54229, dis: 0.69478, time: 0.093
[2, 400] enc: 0.76690, rec: 0.54548, dis: 0.69147, time: 0.093
[2, 500] enc: 0.75384, rec: 0.54181, dis: 0.70151, time: 0.092
[2, 600] enc: 0.72268, rec: 0.55278, dis: 0.68649, time: 0.092
[2, 700] enc: 0.73945, rec: 0.53534, dis: 0.68834, time: 0.091
[2, 800] enc: 0.74159, rec: 0.53488, dis: 0.69479, time: 0.095
[2, 900] enc: 0.71792, rec: 0.54772, dis: 0.69458, time: 0.093
[2, 1000] enc: 0.71855, rec: 0.54943, dis: 0.68657, time: 0.095
[2, 1100] enc: 0.73327, rec: 0.55214, dis: 0.69187, time: 0.081
[2, 1200] enc: 0.73985, rec: 0.54215, dis: 0.69557, time: 0.081
[2, 1300] enc: 0.73616, rec: 0.52775, dis: 0.68827, time: 0.088
[2, 1400] enc: 0.71121, rec: 0.54073, dis: 0.69446, time: 0.091
[2, 1500] enc: 0.74431, rec: 0.53306, dis: 0.69286, 

/home/xyang18/miniconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


...

[86, 0] enc: 0.69695, rec: 0.48899, dis: 0.69346, time: 0.090
[86, 100] enc: 0.69366, rec: 0.50053, dis: 0.69327, time: 0.087
[86, 200] enc: 0.69422, rec: 0.50726, dis: 0.69370, time: 0.091
[86, 300] enc: 0.69309, rec: 0.49910, dis: 0.69323, time: 0.086
[86, 400] enc: 0.69222, rec: 0.51127, dis: 0.69339, time: 0.095
[86, 500] enc: 0.69550, rec: 0.48959, dis: 0.69321, time: 0.089
[86, 600] enc: 0.69143, rec: 0.49739, dis: 0.69364, time: 0.082
[86, 700] enc: 0.69550, rec: 0.50179, dis: 0.69312, time: 0.116
[86, 800] enc: 0.69334, rec: 0.49165, dis: 0.69322, time: 0.087
[86, 900] enc: 0.69325, rec: 0.49027, dis: 0.69269, time: 0.097
[86, 1000] enc: 0.69632, rec: 0.51196, dis: 0.69271, time: 0.093
[86, 1100] enc: 0.69351, rec: 0.49865, dis: 0.69342, time: 0.091
[86, 1200] enc: 0.69240, rec: 0.49554, dis: 0.69269, time: 0.089
[86, 1300] enc: 0.69496, rec: 0.49270, dis: 0.69241, time: 0.092
[86, 1400] enc: 0.69735, rec: 0.50228, dis: 0.69212, time: 0.095
[86, 1500] enc: 0.69262, rec: 0.51122

/home/xyang18/miniconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


[87, 0] enc: 0.69717, rec: 0.49277, dis: 0.69327, time: 0.091
[87, 100] enc: 0.69630, rec: 0.49236, dis: 0.69290, time: 0.093
[87, 200] enc: 0.69559, rec: 0.49265, dis: 0.69322, time: 0.094
[87, 300] enc: 0.69635, rec: 0.51512, dis: 0.69316, time: 0.085
[87, 400] enc: 0.69594, rec: 0.48209, dis: 0.69317, time: 0.099
[87, 500] enc: 0.69400, rec: 0.51647, dis: 0.69289, time: 0.090
[87, 600] enc: 0.69262, rec: 0.49179, dis: 0.69375, time: 0.086
[87, 700] enc: 0.69234, rec: 0.48267, dis: 0.69319, time: 0.091
[87, 800] enc: 0.69629, rec: 0.50101, dis: 0.69359, time: 0.094
[87, 900] enc: 0.69341, rec: 0.50363, dis: 0.69303, time: 0.094
[87, 1000] enc: 0.69379, rec: 0.50698, dis: 0.69302, time: 0.089
[87, 1100] enc: 0.68942, rec: 0.49090, dis: 0.69321, time: 0.093
[87, 1200] enc: 0.69184, rec: 0.48572, dis: 0.69288, time: 0.087
[87, 1300] enc: 0.69481, rec: 0.49641, dis: 0.69272, time: 0.085
[87, 1400] enc: 0.69727, rec: 0.49092, dis: 0.69254, time: 0.083
[87, 1500] enc: 0.69395, rec: 0.49364

/home/xyang18/miniconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


[88, 0] enc: 0.69059, rec: 0.50429, dis: 0.69319, time: 0.091
[88, 100] enc: 0.69286, rec: 0.50444, dis: 0.69281, time: 0.099
[88, 200] enc: 0.69592, rec: 0.48517, dis: 0.69240, time: 0.094
[88, 300] enc: 0.69709, rec: 0.49501, dis: 0.69348, time: 0.086


In [ ]:
if not opts.evalMode:
    eval_mode(dae=dae, exDir=exDir, M=20, testLoader=testLoader, svm=svm)
    svm = train_svm(dae=dae, svm=svm, trainLoader=trainLoader, testLoader=testLoader, exDir=exDir, lr=opts.svmLR)


#Train a linear-SVM classifier on the enocdings